In [1]:
import sys
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score

In [2]:
blind_test = pd.read_csv("Result_3.csv")  



/Users/danieltsinis/Documents/sandbox/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
#create cols 

risk_level = ['RISK_600','RISK_700','RISK_800','RISK_900']
tenure_buckets = ['0-25','51-100','101-200','201-300','301-400','401-500','501-1000','1000+']

final_df = pd.DataFrame()

for risk in risk_level:
    for bucket in tenure_buckets:

        eval_df = blind_test[blind_test['TENURE_BUCKETS']==bucket]
    #     eval_df = blind_test[['TRANSACTION_ID','LABEL','AMOUNT','RISK_600']]

        eval_df_tp_amount = eval_df[(eval_df['LABEL']==1)&(eval_df[risk]==1)]['AMOUNT'].sum()
        eval_df_tn_amount = eval_df[(eval_df['LABEL']==0)&(eval_df[risk]==0)]['AMOUNT'].sum()
        eval_df_fp_amount = eval_df[(eval_df['LABEL']==0)&(eval_df[risk]==1)]['AMOUNT'].sum()
        eval_df_fn_amount = eval_df[(eval_df['LABEL']==1)&(eval_df[risk]==0)]['AMOUNT'].sum()

        CM = confusion_matrix(eval_df['LABEL'], eval_df[risk])
        TN = CM[0][0]
        FN = CM[1][0]
        TP = CM[1][1]
        FP = CM[0][1]
        
        d = [{'Risk': risk,
              'Bucket':str(bucket),
              'Size': str(len(eval_df)),
              'Accuracy':accuracy_score(eval_df['LABEL'],eval_df[risk]),
              'Precision':precision_score(eval_df['LABEL'],eval_df[risk]),
              'Recall':recall_score(eval_df['LABEL'],eval_df[risk]),
              'False Positive Rate': (FP/(FP+TN)),
              
              'Number of True Negatives': TN,
              'Number of True Positives': TP,
              'Number of False Negatives': FN,
              'Number of False Negatives': FP,
              'TP Cost':eval_df_tp_amount,
              'TN Cost':eval_df_tn_amount,
              'FP Cost':eval_df_fp_amount,
              'FN Cost':eval_df_fn_amount
             }]
  
        iter_df = pd.DataFrame(d)
        final_df = final_df.append(iter_df)

In [19]:
final_df.to_csv('amount_2.csv')